# Nagel–Schreckenberg model

Nagel–Schreckenberg model is a simple model used for modelling highway traffic.
#### Road
A road consists of cells. Each cell represents a single car or empty space on the road. 
#### Car 
Each car has a velocity (originally 0-5 units).
#### Transisition function
The transition function performs the following steps (order matters):
* Acceleration: The velocity of all cars having a velocity lower than the maximum velocity is increased by 1.
* Slowing down: If the distance between the current and the following car is smaller than the car's velocity, the velocity is reduced to the number of empty cells in front of the car (avoiding collisions).
* Randomization: The speed of all cars that have a *velocity >= 1* is reduced by one unit with the probability of *p*. 
* Car motion: All cars are moved forward by the *velocity* cells.

(source and more detailed description: [here](https://en.wikipedia.org/wiki/Nagel–Schreckenberg_model))

## Python implementation of Nagel-Schreckenberg model (6 points)

* Create a working Nagel-Schreckenberg model using the following template (4 points).
* Add periodic boundaries (1 point).
* Add a function that will randomly add new cars at the beginning of the road (1 point).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

In [2]:
%%capture
# Parameters
roadLength = 50
minVelocity = 0
maxVelocity = 5

# Number of steps 
steps = 100

rand_probability = 0.5 

# Config
fig, ax = plt.subplots(figsize=(12,4))
plt.axis('off')

In [3]:
class Car:
    distance = 0

    
    def __init__(self, minVelocity, maxVelocity):
        import random
        self.minVelocity = minVelocity
        self.maxVelocity = maxVelocity
        self.color = random.randint(0,200)
        self.velocity = random.randint(minVelocity,maxVelocity)
    
    #TODO Implement increaseVelcoity and randomizeVelocity functions 
    def increaseVelocity(self,maxVelocity):
        if(self.velocity < maxVelocity):
            self.setVelocity(self.velocity+1, self.maxVelocity)
    
    def setVelocity(self, newVelocity, maxVelocity):
        if newVelocity <= maxVelocity:
            self.velocity = newVelocity
        
    def randomizeVelocity(self, probability):
        if (np.random.choice([1,0], 1, p=[probability, 1-probability])):
            if(self.velocity>0):
                self.setVelocity(self.velocity-1, self.maxVelocity)
            
    def getColor(self):
        return self.color
    
    def isEmpty(self):
        return false
    
    def setDistance(self,distance):
        self.distance = distance
        
    def breakBasedOnSituation(self, i, grid, roadLength):
        for j in range(1, self.velocity+1):
            if(type(grid[(i+j)%roadLength]) is Car):
                self.setVelocity(j-1, self.maxVelocity)
                break
    
    def move(self, index, newGrid, roadLength):
        newGrid[(index+self.velocity)%roadLength] = self

class Road:
    def __init__(self):
        return
    def getColor(self):
        return 255
    def isEmpty(self):
        return true

In [4]:
def initGrid(roadLength):
    # TODO Randomly initialize grid here (remove existing code)
    # grid should be a list

    grid = np.random.choice([1,0], roadLength, p=[0.1,0.9])
    road = []
    for i in range(0, roadLength):
        if(grid[i]==1):
            road.append(Car(minVelocity, maxVelocity))
        else:
            road.append(Road())
    return road

def show(grid, im):
        im.set_data(np.array(list(map(lambda x : x.getColor(), grid))).reshape(1,roadLength))

        
def add_car(grid, probability):
    if(type(grid[0]) is Road):
        if(np.random.choice([1,0], 1, p=[probability,1-probability])):
            grid[0] = Car(minVelocity, maxVelocity)

def update(frameNum, grid, im, roadLength):
    
    if(frameNum == 0):
        show(grid,im)
        
    else:


        # 1.Acceleration
        for i in grid:
            if type(i) is Car:
                i.increaseVelocity(maxVelocity)

        # 2. Slow down
        for index, item in enumerate(grid):
            if type(item) is Car:
                item.breakBasedOnSituation(index, grid, roadLength)

        # TODO Implement slow down


        # 3. Randomization
        for i in grid:
            if type(i) is Car:
                i.randomizeVelocity(rand_probability)


        # 4. Car motion
        newGrid = [Road()] * roadLength
        for index, item in enumerate(grid):
            if type(item) is Car:
                item.move(index, newGrid, roadLength)

        # TODO Implement motion

        grid[:] = newGrid[:] 
        
        add_car(grid, 0.1)
        
        show(grid, im)
    return im,



In [5]:
grid = initGrid(roadLength)
im = ax.imshow(np.array(list(map(lambda x : x.getColor(), grid))).reshape(1,roadLength),cmap = 'gist_ncar')
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [grid, im, roadLength])

HTML(ani.to_jshtml())


